In [ ]:
import os
import numpy as np
import glob
import shutil
import dlib
import time
import cv2
from scipy.spatial import distance as dist

## Tinh khau do mieng va mat

In [ ]:
# đầu vào mắt trái [36,41] mắt phải [42,47]
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1],eye[5])
    B = dist.euclidean(eye[2],eye[4])
    C = dist.euclidean(eye[0],eye[3])
    ear = (A+B)/(2.0*C)
    return ear

# đầu vào các điểm [60-67]
def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[1],mouth[7])
    B = dist.euclidean(mouth[2],mouth[6])
    C = dist.euclidean(mouth[3],mouth[5])
    D = dist.euclidean(mouth[0],mouth[4])
    mar = (A+B+C)/(2.0*D)
    return mar

## Danh nhan cho 1 anh

In [ ]:
def label_img(img_path):
    EAR = 0
    MAR = 0
    start_time = time.time()
    img = cv2.imread(img_path)
    detector = dlib.get_frontal_face_detector() # Load face detector
    path_landmark = "./shape_predictor_68_face_landmarks.dat"
    predictor = dlib.shape_predictor(path_landmark)
    dets = detector(img,upsample_num_times=0)
    for d in dets:
        shape = predictor(img, d)

        # tọa độ mắt trái (indexes 36 - 41)
        left_eye_landmarks = [(shape.part(i).x, shape.part(i).y) for i in range(36, 42)]

        # tọa độ mắt phải (indexes 42 - 47)
        right_eye_landmarks = [(shape.part(i).x, shape.part(i).y) for i in range(42, 48)]
        
        # lấy tọa độ các điểm cho miệng
        mouth_landmark = [(shape.part(i).x, shape.part(i).y) for i in range(60, 68)]

        # tính ear
        ear_l = eye_aspect_ratio(left_eye_landmarks)
        ear_r = eye_aspect_ratio(right_eye_landmarks)
        EAR = (ear_l + ear_r)/2.0

        # Tinsh mieenjg
        MAR = mouth_aspect_ratio(mouth_landmark)


    
    end_time = time.time() - start_time
    print("Time: ",end_time)
    print("EAR: ",EAR)
    print("MAR: ",MAR)
    return EAR,MAR

## Danh nhan cho tat ca anh

In [ ]:
folder_path = 'frame_from_video'
yawn_folder = 'yawn'
microsleep_folder = 'microsleep'
normal_folder = 'normal'

# Tìm tất cả các tệp .jpg trong thư mục và các thư mục con
jpg_files = glob.glob(os.path.join(folder_path, '**', '*.jpg'), recursive=True)


for file_path in jpg_files:
    EAR, MAR = label_img(file_path)
    if EAR >= 0.5:
        shutil.move(file_path, os.path.join(yawn_folder, file_path))
    if EAR <= 0.2 and MAR <= 0.5:
        shutil.move(file_path, os.path.join(microsleep_folder, file_path))
    if MAR < 0.5 and EAR > 0.2:
        shutil.move(file_path, os.path.join(normal_folder, file_path))